# 01 - Exploración de Datos

Este notebook explora el dataset MATH y el dataset de física,
mostrando estadísticas, distribuciones y ejemplos.

In [4]:
import json
import os
import sys
import numpy as np

# Agregar directorio raíz al path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Directorio del proyecto: {project_root}")

Directorio del proyecto: /home/melissa/transformer_math_physics_tutor


## 1. Cargar Datasets

In [ ]:
# Cargar datasets
math_path = os.path.join(project_root, 'data', 'math_clean.json')
physics_path = os.path.join(project_root, 'data', 'physics_problems.json')

datasets = {}

if os.path.exists(math_path):
    with open(math_path, 'r', encoding='utf-8') as f:
        datasets['math'] = json.load(f)
    print(f"Dataset Matemáticas cargado: {len(datasets['math'])} problemas")
else:
    print(f"No se encontró {math_path}")

# Cargar dataset de física
if os.path.exists(physics_path):
    with open(physics_path, 'r', encoding='utf-8') as f:
        datasets['physics'] = json.load(f)
    print(f"Dataset Física cargado: {len(datasets['physics'])} problemas")
else:
    print(f"No se encontró {physics_path}")

Dataset Matemáticas cargado: 215 problemas
Dataset Física cargado: 20 problemas


## 2. Estadísticas de Longitud

In [9]:
def compute_length_stats(problems, label="Dataset"):
    """Calcula y muestra estadísticas de longitud."""
    prob_lengths = [len(p['problem'].split()) for p in problems]
    sol_lengths = [len(p['solution'].split()) for p in problems]
    prob_char_lengths = [len(p['problem']) for p in problems]
    sol_char_lengths = [len(p['solution']) for p in problems]
    
    print(f"\n{'='*50}")
    print(f"Estadísticas de {label} ({len(problems)} problemas)")
    print(f"{'='*50}")
    
    print(f"\nLongitud del PROBLEMA (palabras):")
    print(f"  Media: {np.mean(prob_lengths):.1f}")
    print(f"  Mediana: {np.median(prob_lengths):.1f}")
    print(f"  Min: {np.min(prob_lengths)}, Max: {np.max(prob_lengths)}")
    print(f"  Std: {np.std(prob_lengths):.1f}")
    
    print(f"\nLongitud de la SOLUCIÓN (palabras):")
    print(f"  Media: {np.mean(sol_lengths):.1f}")
    print(f"  Mediana: {np.median(sol_lengths):.1f}")
    print(f"  Min: {np.min(sol_lengths)}, Max: {np.max(sol_lengths)}")
    print(f"  Std: {np.std(sol_lengths):.1f}")
    
    print(f"\nLongitud del PROBLEMA (caracteres):")
    print(f"  Media: {np.mean(prob_char_lengths):.1f}")
    print(f"  Mediana: {np.median(prob_char_lengths):.1f}")
    print(f"  Min: {np.min(prob_char_lengths)}, Max: {np.max(prob_char_lengths)}")
    
    print(f"\nLongitud de la SOLUCIÓN (caracteres):")
    print(f"  Media: {np.mean(sol_char_lengths):.1f}")
    print(f"  Mediana: {np.median(sol_char_lengths):.1f}")
    print(f"  Min: {np.min(sol_char_lengths)}, Max: {np.max(sol_char_lengths)}")
    
    return prob_lengths, sol_lengths, prob_char_lengths, sol_char_lengths

# Calcular estadísticas
stats = {}
for name, data in datasets.items():
    stats[name] = compute_length_stats(data, name.upper())


Estadísticas de MATH (215 problemas)

Longitud del PROBLEMA (palabras):
  Media: 7.4
  Mediana: 6.0
  Min: 3, Max: 18
  Std: 3.3

Longitud de la SOLUCIÓN (palabras):
  Media: 2.1
  Mediana: 1.0
  Min: 1, Max: 4
  Std: 1.2

Longitud del PROBLEMA (caracteres):
  Media: 24.1
  Mediana: 20.0
  Min: 9, Max: 65

Longitud de la SOLUCIÓN (caracteres):
  Media: 4.7
  Mediana: 4.0
  Min: 1, Max: 14

Estadísticas de PHYSICS (20 problemas)

Longitud del PROBLEMA (palabras):
  Media: 17.6
  Mediana: 18.0
  Min: 13, Max: 23
  Std: 3.3

Longitud de la SOLUCIÓN (palabras):
  Media: 11.6
  Mediana: 12.0
  Min: 8, Max: 16
  Std: 2.4

Longitud del PROBLEMA (caracteres):
  Media: 81.9
  Mediana: 80.5
  Min: 58, Max: 111

Longitud de la SOLUCIÓN (caracteres):
  Media: 38.9
  Mediana: 35.5
  Min: 21, Max: 64


## 3. Distribución por Temas

In [7]:
from collections import Counter

for name, data in datasets.items():
    print(f"\n--- Distribución de temas en {name.upper()} ---")
    topics = Counter(p.get('topic', 'unknown') for p in data)
    for topic, count in topics.most_common():
        bar = '█' * (count // max(1, max(topics.values()) // 30))
        print(f"  {topic:25s}: {count:4d} {bar}")
    
    levels = Counter(p.get('level', 0) for p in data)
    print(f"\n  Niveles de dificultad:")
    for level, count in sorted(levels.items()):
        print(f"    Nivel {level}: {count} problemas")


--- Distribución de temas en MATH ---
  unknown                  :  215 ██████████████████████████████

  Niveles de dificultad:
    Nivel 0: 215 problemas

--- Distribución de temas en PHYSICS ---
  kinematics               :    4 ████
  electricity              :    4 ████
  dynamics                 :    3 ███
  energy                   :    3 ███
  thermodynamics           :    2 ██
  oscillations             :    1 █
  waves                    :    1 █
  statics                  :    1 █
  fluids                   :    1 █

  Niveles de dificultad:
    Nivel 1: 20 problemas


## 4. Visualización de Distribuciones

In [12]:
# Visualización con texto (sin dependencia de matplotlib)
def text_histogram(data, bins=20, title="Histograma", width=50):
    """Crea un histograma de texto."""
    if not data:
        print("Sin datos para visualizar.")
        return
    
    min_val = min(data)
    max_val = max(data)
    bin_width = max(1, (max_val - min_val) / bins)
    
    counts = [0] * bins
    for val in data:
        idx = min(int((val - min_val) / bin_width), bins - 1)
        counts[idx] += 1
    
    max_count = max(counts)
    
    print(f"\n{title}")
    print(f"{'─' * (width + 20)}")
    
    for i, count in enumerate(counts):
        if count > 0:
            edge = min_val + i * bin_width
            bar_len = int(count / max(max_count, 1) * width)
            bar = '█' * bar_len
            print(f"  {edge:6.0f} - {edge+bin_width:6.0f} | {bar} ({count})")

# Visualizar distribuciones
for name, (pl, sl, pcl, scl) in stats.items():
    text_histogram(pl, title=f"{name.upper()} - Longitud del problema (palabras)")
    text_histogram(sl, title=f"{name.upper()} - Longitud de la solución (palabras)")
    text_histogram(pcl, title=f"{name.upper()} - Longitud del problema (caracteres)")
    text_histogram(scl, title=f"{name.upper()} - Longitud de la solución (caracteres)")


MATH - Longitud del problema (palabras)
──────────────────────────────────────────────────────────────────────
       3 -      4 | █████ (9)
       5 -      6 | ██████████████████████████████████████████████████ (85)
       6 -      7 | █████████████████ (29)
       7 -      8 | ████████ (14)
       8 -      9 | █████████████████ (29)
       9 -     10 | ██ (5)
      12 -     13 | ████ (7)
      13 -     14 | ██████████████████ (31)
      14 -     15 |  (1)
      15 -     16 | █ (3)
      18 -     19 | █ (2)

MATH - Longitud de la solución (palabras)
──────────────────────────────────────────────────────────────────────
       1 -      2 | ██████████████████████████████████████████████████ (111)
       3 -      4 | ██████████████████████████████ (67)
       4 -      5 | ████████████████ (37)

MATH - Longitud del problema (caracteres)
──────────────────────────────────────────────────────────────────────
       9 -     12 | ██████████████████████████████████████████████████ (51)
      

## 5. Ejemplos de Problemas y Soluciones

In [13]:
for name, data in datasets.items():
    print(f"\n{'='*60}")
    print(f"EJEMPLOS DE {name.upper()}")
    print(f"{'='*60}")
    
    # Mostrar primeros 5 problemas
    for i, prob in enumerate(data[:5], 1):
        print(f"\n--- Ejemplo {i} ---")
        print(f"Tema: {prob.get('topic', 'N/A')} | Nivel: {prob.get('level', 'N/A')}")
        print(f"Problema: {prob['problem'][:200]}")
        print(f"Solución: {prob['solution'][:200]}")


EJEMPLOS DE MATH

--- Ejemplo 1 ---
Tema: N/A | Nivel: N/A
Problema: 1 + 1 = ?
Solución: 2

--- Ejemplo 2 ---
Tema: N/A | Nivel: N/A
Problema: 2 + 3 = ?
Solución: 5

--- Ejemplo 3 ---
Tema: N/A | Nivel: N/A
Problema: 5 + 7 = ?
Solución: 12

--- Ejemplo 4 ---
Tema: N/A | Nivel: N/A
Problema: 8 + 9 = ?
Solución: 17

--- Ejemplo 5 ---
Tema: N/A | Nivel: N/A
Problema: 10 + 15 = ?
Solución: 25

EJEMPLOS DE PHYSICS

--- Ejemplo 1 ---
Tema: kinematics | Nivel: 1
Problema: A car travels at 60 km/h for 2 hours. What distance does it cover?
Solución: d = v * t = 60 * 2 = 120 km

--- Ejemplo 2 ---
Tema: dynamics | Nivel: 1
Problema: What is the net force on a 5 kg object with acceleration 3 m/s^2?
Solución: F = m * a = 5 * 3 = 15 N

--- Ejemplo 3 ---
Tema: kinematics | Nivel: 1
Problema: An object falls freely from 20 m height. How long does it take to reach the ground? (g = 10 m/s^2)
Solución: h = (1/2)*g*t^2, t = sqrt(2*h/g) = sqrt(2*20/10) = sqrt(4) = 2 s

--- Ejemplo 4 ---
Tema: energy | Niv

## 6. Análisis del Vocabulario de Caracteres

In [14]:
# Analizar caracteres únicos en el dataset
for name, data in datasets.items():
    all_text = ' '.join(p['problem'] + ' ' + p['solution'] for p in data)
    char_freq = Counter(all_text)
    
    print(f"\n--- Vocabulario de caracteres ({name.upper()}) ---")
    print(f"Total de caracteres: {len(all_text):,}")
    print(f"Caracteres únicos: {len(char_freq)}")
    print(f"\nTop 30 caracteres más frecuentes:")
    
    for char, count in char_freq.most_common(30):
        display_char = repr(char) if char in ' \t\n' else char
        pct = count / len(all_text) * 100
        print(f"  '{display_char}': {count:6d} ({pct:.1f}%)")
    
    # Tamaño estimado del vocabulario del tokenizer
    print(f"\nTamaño estimado del tokenizer: {len(char_freq) + 4} tokens")
    print(f"  ({len(char_freq)} caracteres + 4 tokens especiales)")


--- Vocabulario de caracteres (MATH) ---
Total de caracteres: 6,614
Caracteres únicos: 57

Top 30 caracteres más frecuentes:
  '' '':   2042 (30.9%)
  '=':    295 (4.5%)
  'e':    270 (4.1%)
  '0':    243 (3.7%)
  't':    215 (3.3%)
  'a':    202 (3.1%)
  '1':    189 (2.9%)
  'o':    185 (2.8%)
  'h':    173 (2.6%)
  'i':    169 (2.6%)
  '2':    168 (2.5%)
  'f':    164 (2.5%)
  'x':    156 (2.4%)
  'r':    148 (2.2%)
  '5':    144 (2.2%)
  'n':    128 (1.9%)
  'd':    119 (1.8%)
  's':    118 (1.8%)
  '?':    117 (1.8%)
  '3':    106 (1.6%)
  '4':     90 (1.4%)
  '/':     80 (1.2%)
  'v':     79 (1.2%)
  'l':     78 (1.2%)
  'm':     77 (1.2%)
  'W':     59 (0.9%)
  '6':     57 (0.9%)
  'S':     52 (0.8%)
  ':':     52 (0.8%)
  'g':     50 (0.8%)

Tamaño estimado del tokenizer: 61 tokens
  (57 caracteres + 4 tokens especiales)

--- Vocabulario de caracteres (PHYSICS) ---
Total de caracteres: 2,455
Caracteres únicos: 64

Top 30 caracteres más frecuentes:
  '' '':    584 (23.8%)
  'e':